In [ ]:
from postprocessing import *

In [ ]:
simulation_directory = str('/n/holyscratch01/hernquist_lab/borodina/turb_drive_cooling/turb')
output_directory = simulation_directory+"/output/"
figures_directory = simulation_directory + "/output/figures/"

## Mach number

In [ ]:
turb = pd.read_csv(simulation_directory + '/output/turb.txt', sep=' ', names=['t',
                                                           'mach',
                                                           'etot/mass', 
                                                           'dudt/mass_drive', 
                                                           'dudt/mass_diss', 
                                                           'energyinjected/mass', 
                                                           'energydissipated/mass'])

In [ ]:
mpl.rcParams['figure.dpi']= 200

fig, ax = plt.subplots(figsize=(5,3))
ax.plot(turb['t'] * unit_time_in_megayr, turb['mach'])
ax.set_xlabel('t [Myr]', fontsize=15)
ax.set_ylabel('M', fontsize=15)
plt.show()

## density in the center

In [ ]:
simulation_directory = str('/n/holyscratch01/hernquist_lab/borodina/turb_drive_cooling_dense/turb')
output_directory = simulation_directory+"/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
central_density = []

In [ ]:
i_file = -1  # skip snap 0
while True:
    i_file += 1
    filename = "snap_%03d.hdf5" % (i_file)
    try:
        snap_data = h5py.File(output_directory + filename, "r")
    except:
        break
    time = get_time_from_snap(snap_data) * unit_time_in_megayr
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    mask = np.sqrt((x - 500) ** 2 + (y - 500) ** 2 + (z - 500) ** 2) < 100
    n = np.mean(snap_data['PartType0/Density'][mask]) * unit_density
    central_density.append([time, n])

In [ ]:
central_density = np.array(central_density).T

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(central_density[0], central_density[1])
ax.scatter(central_density[0][12], central_density[1][12],c='red', zorder=100)
ax.scatter(central_density[0][20], central_density[1][20],c='red', zorder=100)
ax.scatter(central_density[0][28], central_density[1][28],c='red', zorder=100)
ax.set_xlabel('t [Myr]', fontsize=12)
ax.set_ylabel('mean density inside 100pc [g cm$^{-3}$]', fontsize=10)
plt.show()